In [1]:
import pickle
#loading tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [2]:
# loading our models
from keras.models import load_model
skill_prediction_model = load_model('skill_prediction.h5')
job_predictor_model = load_model('job_predictor.h5')

In [3]:
# loading vectorizer and tokenizer

with open('vectorizer.pickle', 'rb') as handle:
    vectorizer = pickle.load(handle)

with open('LabelEncoder.pickle', 'rb') as handle:
    LabelEncoder = pickle.load(handle)


In [4]:
from keras_preprocessing.sequence import pad_sequences
def predict(model,tokenizer,word):
    input = pad_sequences(tokenizer.texts_to_sequences([word]),maxlen=20)
    # input = tokenizer.texts_to_sequences([word])
    d = model.predict(input)
    # print('probebilty ',d)
    if d <= 0.5:
        d = 0 #'non skill word'
    else:
        d = 1 #'skill word' 
    return d

In [5]:
import nltk
import spacy
import string
import re
#Initializing the spacy engine and then use it to process the text:
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
stop_words = set(stopwords.words("english"))
# remove stopwords function
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    r = TreebankWordDetokenizer().detokenize(filtered_text)
    return r

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def text_preprocess(text):
    r = text.lower() # coverting into lower case
    r = remove_stopwords(r) # removing stop words
    r = re.sub(r'\d+', '', r) # removing numbers
    r = remove_punctuation(r) # removing punctuations
    r = " ".join(r.split()) # removing white sapce
    r = ''.join(r.splitlines()) # removing newline character
    return r

In [15]:
import numpy as np

def predict_title(text):
    nouns = []
    doc = nlp(text.lower())
    for chunk in doc.noun_chunks:
        chunk = text_preprocess(str(chunk))
        nouns.append(chunk)
    dir_temp = dict()
    for i in nouns:
        # print(type(i))
        r = predict(skill_prediction_model,tokenizer,i)
        if r == 1:
          dir_temp[i] = r
    print(list(dir_temp.keys()))
    data = str(list(dir_temp.keys()))
    print(type(data))
    vec_data = vectorizer.transform([data])
    pred = job_predictor_model.predict(vec_data)
    print(pred)
    title = np.argmax(pred,axis=1)

    title = LabelEncoder.inverse_transform(title)
    return 'Your most suitable job title is ', title
   

    
    

In [13]:
text = '''Vrajeshkumar Bhatt
Data Scientist
Skilled team player with a strong background in Python, Data
Analytics, and Machine Learning. Have experience with ml model
training and deployment. Works well independently to handle projects
as well as a great team player. Quick learner with good communication
skills, dependable, and ready to learn and grow.
vrajeshbhatt113@gmail.com
+1 4379712456
Toronto, canada
linkedin.com/in/vrajeshbhatt
github.com/vrajeshbhatt
WORK EXPERIENCE
ML Intern
GatiSofTech
01/2021 - 04/2021, Surat, India
Software Development Company
Manage the planning and development of design and procedures
for the project Ticket Sampling. Designed the solution, data gathering, and implemented the
machine learning solutions. Gathered, analyzed, and prepared the data for development. Worked with Multi-target Neural Network model for prediction
and classification with python. Deployed model with Flask framework on the web platform. Made Auto Rebuild ML model that can rebuild itself and with new
data
Python Back-end Engineer
Upugo Fitness & Sports
12/2021 - 03/2022, Bangalore, India
Responsible for writing scalable backend applications in Python. Coordinating with Front-end Team. Built applications in Python, Flask, and JavaScript. Worked on Flask- Framework, and deployment of exercises and
games and REST services for the front end. Communicate with the team and integrate feedback into project
guidelines
EDUCATION
Bachelor of Technology
PP Savani University
08/2017 - 07/2021, 8.51 CGPA
Computer Engineering
Two-year Ontario College Graduate Certificate
Loyalist College in Toronto
05/2022 - Present,
Artificial Intelligence and Data Science
TECHNICAL SKILLS
Python Flask SQL Data Analytics
Machine Learning Tableau
Data Preparation ANN
ML model deployment Heroku Git
Web Development JavaScript
PERSONAL PROJECTS
Upasthiti: A Speaker Recognition System
Identifying users using the characteristics extracted from
their voices.
Implemented using CNN and MFCC feature extraction. Trained model with own dataset and achieved 78% validation
accuracy.
Facial Expression Recognition
Recognize expressions from faces such as natural, happy, angry, surprised, and sad.
It is created using CNN, Keras, OpenCV, Flask. Deployed this project on Web using Flask and Heroku.
Job Title Prediction using Machine Learning
Predicts the most suitable job title based on their personality
trait, interest, and skills. Model build with artificial neural network with own data set which has 15 job titles and 50 attributes and focused on IT
related fields
ONLINE COURSES
Data Science Hands-On with Open Source Tools
Cognitive Class an IBM Initiative - issued on 06/2019
Machine Learning with Python
Cognitive Class an IBM Initiative - issued on 03/2020
Neural Networks and Deep Learning
Coursera - issued on 05/2020
SQL Masterclass: SQL for Data Analytic'''

In [20]:
o = predict_title(text)
print(o)

1/1 [==============================] - 0s 39ms/step
['agile methodologies', 'ai', 'data', 'pure software', 'skills \uf0d8 technical manual testing automation testing microsoft office suite selenium', 'java python hp qc', 'soap ui', 'development', 'algorithms', 'software systems', 'code', 'analysis', 'impact analysis', 'data driven framework', 'test automation framework libraries', 'cucumber framework', 'integration issues', '• machine learning']
<class 'str'>
1/1 [==============================] - 0s 36ms/step
[[0.08158923 0.65544564 0.6205959  0.4697493  0.49769154]]
('Your most suitable job title is ', array(['Data Analyst'], dtype=object))


In [17]:
len(pred)

135

In [26]:
d = {'a':1,'b':1}

In [52]:
str(list(d.keys()))

"['a', 'b']"

In [56]:
vectorizer.transform(["this is the post for the data science"])

<1x803 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [10]:
LabelEncoder.classes_

array(['Cloud Engineer', 'Data Analyst', 'Database Administrator',
       'Full Stack Developer', 'machine learning engineer'], dtype=object)